In [1]:
#Archangel 0.1.0

#Importação dos pacotes usados

#Classe AVB
import numpy as np
import sounddevice as sd
import pandas as pd
import itertools as it
from scipy import signal as sg

#conda install rpy2 #instalador do rpy2
#rpy2.robjects.r('install.packages("dtw")') #instalador do pacote dtw do R. Rodar somente uma vez
import rpy2.robjects.numpy2ri
from rpy2.robjects.packages import importr
from rpy2.robjects.functions import SignatureTranslatedFunction
rpy2.robjects.numpy2ri.activate()
R = rpy2.robjects.r
DTW = importr('dtw')
rpy2.robjects.r('''
                    dtw_dist <- function(MFCC1, MFCC2) {
                    return(dtw(MFCC1,MFCC2)$normalizedDistance)
                    }
                    ''')
dtw_dist = rpy2.robjects.globalenv['dtw_dist']

#Classe GUI
import winsound
from tkinter import *

In [ ]:
#Variáveis globais

GV = {"TA" : 8000, #Taxa de Amostragem 
      "TH": 0.13, #Threshold
      "TR": 1*3*8000, #Time Record
      "TS": 1, #Time Shift
      "fv":[0.00625*2,0.425*2],
      "ordem_fbw": 8,
      "rep_rle": 100,
      "precisao": 1.3}

AVB_RS = {} #Este é o dicionário onde estarão todos as vozes de originais
AVB_PS = {} #Este é o dicionário onde estarão todas as vozes de treino processadas
AVB_MFCC = {} #Este é o dicionário onde estarão os MFCCs

class ASP: #classe ASP - Audio Signal Processor       
    def rsp(self,audio): #Função reshape
        n_colunas = int(GV["TA"]*GV["TS"])
        zeros_add = len(audio)%n_colunas
        zeros_add = int(n_colunas - zeros_add)
        zeros_add = np.zeros(zeros_add)
        audio = np.concatenate((audio,zeros_add), axis = None)
        n_linhas = int(len(audio)/n_colunas)
        return audio.reshape((n_linhas, n_colunas))
    
    def bwf(self,audio): #Função Filtro ButterWorth
        f1, f2 = sg.butter(GV["ordem_fbw"], GV["fv"], btype = 'bandpass')
        return sg.lfilter(f1, f2, audio)
    
    def offset(self,audio): #Função offset
        media = np.mean(audio)
        return (audio - media)
    
    def norm(self,audio): #Função normalização
        audio[np.absolute(audio/np.max(audio)) < GV["TH"]] = 0 
        return audio
    
    def rle(self,audio): #Função RLE
        audio_rle = []
        for i,j in it.groupby(audio):
            b = list(j)
            if len(b) > GV["rep_rle"] and i == 0: pass
            else: audio_rle = audio_rle + list(it.repeat(i,len(b)))
        return audio
    
    def mfcc(self,audio): #Função MFCC
        if len(audio) > (GV["TA"]/2): #Este é o frame, não o áudio todo
            from python_speech_features import mfcc
            mfcc = mfcc(audio,GV["TA"])
            (n_linhas, n_colunas) = mfcc.shape
            return list(mfcc.reshape((1,n_linhas*n_colunas))[0])
        else: return [0]
        
    def dtw(self,mfcc1,mfcc2): #Função DTW
        dist = dtw_dist(mfcc1,mfcc2)
        return dist  
        
class AVB(ASP): #Classe AVB - Archangel Voice Biometrics, filha da classe ASP
    def reset_db(self):
        AVB_RS.clear()
        print('Base AVB_RS anulada')
        AVB_PS.clear()
        print('Base AVB_PS anulada')
        AVB_MFCC.clear()
        print('Base AVB_MFCC anulada')
        return 'Bases anuladas'
    
    def get_db(self):
        try:
            lista_cdb = list(AVB_MFCC.keys())
            return lista_cdb
        except:
            return 'Banco de dados de vozes vazio'
    def register(self,audio,ID=''):
        AVB_RS[ID] = audio
        frame = self.rsp(audio) #Quebra do áudio
        id_mfcc = []
        for a in range(len(frame)):
            #Pré-processamento do áudio
            audio = self.bwf(frame[a])
            audio = self.offset(audio)
            audio = self.norm(audio)
            audio = self.rle(audio)
            AVB_PS[ID + str(a)] = audio
            if sum(audio) != 0.0:
                #MFCC
                mfcc_audio = self.mfcc(audio)
                id_mfcc.append(mfcc_audio)
                #print('frame %s cadastrado' %a)
        if id_mfcc == []: return 'Não há áudio suficiente'
        else:
            AVB_MFCC[ID] = id_mfcc
            #print('frames adicionados no AVB_MFCC')
            #return 'MFCC cadastrado com ID %s, com %d blocos de mfcc' % (ID,len(id_mfcc))
            return 'CLIENTE %s CADASTRADO' % ID
    def finder(self,audio,ID=''):
        AVB_RS[ID + 'test'] = audio
        frame = self.rsp(audio) #Quebra do áudio
        id_mfcc = []
        dist = []
        list_mfcc = AVB_MFCC[ID]
        for a in range(len(frame)):
            #Pré-processamento do áudio
            audio = self.bwf(frame[a])
            audio = self.offset(audio)
            audio = self.norm(audio)
            audio = self.rle(audio)
            AVB_PS[ID + str(a) + 'test'] = audio
            if sum(audio) != 0.0:
                #MFCC
                mfcc_audio = self.mfcc(audio)
                id_mfcc.append(mfcc_audio)
                for b in range(len(list_mfcc)):
                    c = self.dtw(audio,list_mfcc[b])
                    dist.append(c)
                    #print('Distância entre frame %s e histórico %d:' % (a,b))
                    #print(c)
        AVB_MFCC[ID + 'test'] = id_mfcc
        if np.min(dist) < GV["precisao"]: return 'CLIENTE IDENTIFICADO!'
        else: return 'Usuário NÃO identificado! Distância média: %s' % np.mean(dist)

class Application(AVB): #Classe GUI (interface gráfica), filha da classe AVB
    def __init__(self,master=AVB):
        self.fontePadrao = ("Arial", "12") #define uma fonte padrão

        #Título superior
        self.L1 = Frame() #L1: linha superior da caixa
        self.L1.pack()
        self.titulo = Label(self.L1, text="!!!Bem vindo ao sistema Archangel!!!", font = self.fontePadrao)
        self.titulo.pack()
        
        #Caixa de texto (local onde o usuário vai colocar seu nome/ID)
        self.L2 = Frame() #L2: segunda linha da caixa
        self.L2.pack()
        self.L2["pady"] = 10 #Abre um espacinho aí...
        self.L2["padx"] = 0
        self.menu = Label(self.L2,text="Nome", font=self.fontePadrao)
        self.txtbox = Entry(self.L2,width = 40,font = self.fontePadrao)
        self.menu.pack(side = LEFT)
        self.txtbox.pack(side = LEFT)
        
        #Botões
        self.L3 = Frame() #L3: terceira linha da caixa, com os botões
        self.L3.pack()
        self.L3["pady"] = 10 #Abre um espacinho aí...
        self.reset_db_button = Button(self.L3, text = "Limpar Registros", bd = '5',font = self.fontePadrao,
                               command = self.reset_db)
        self.reset_db_button.pack(side=RIGHT)
        self.get_db_button = Button(self.L3, text = "Mostrar Registros", bd = '5',font = self.fontePadrao,
                               command = self.get_db)
        self.get_db_button.pack(side=LEFT)
        self.register_button = Button(self.L3, text = "Novo cadastro", bd = '5',font = self.fontePadrao,
                               command = self.rec_register)
        self.register_button.pack(side=LEFT)
        self.finder_button = Button(self.L3, text = "Verificar voz", bd = '5',font = self.fontePadrao,
                               command = self.rec_finder)
        self.finder_button.pack(side=LEFT)
        
       #Respostas (espaço inferior para as respostas dos comandos)
        self.L4 = Frame() #L4: quarta linha da caixa
        self.L4.pack()
        self.answer = Label(self.L4,text="Escolha uma das opções", bd = '5', font=self.fontePadrao)
        self.answer.pack()      
    
    def rec(self):
        winsound.Beep(1800, 700)
        audio = sd.rec(GV["TR"], samplerate=GV["TA"], channels=1)
        sd.wait()
        audio_list = []
        for a in range(GV["TR"]): audio_list.append(audio[a][0])
        return list(audio_list)
        
    def rec_register(self):
        self.answer["text"] = "Ao som do beep diga seu nome"
        audio = self.rec()
        while(float(np.amax(audio)) < GV["TH"]):
            self.answer["text"] = "Tente novamente. Fale mais alto!"
            audio = self.rec()
        ID = self.txtbox.get()
        self.answer["text"] = self.register(audio,ID)
                
    def rec_finder(self):
        self.answer["text"] = "Ao som do beep diga seu nome"
        audio = self.rec()
        while(float(np.amax(audio)) < GV["TH"]):
            self.answer["text"] = "Tente novamente. Fale mais alto!"
            audio = self.rec()
        ID = self.txtbox.get()
        self.answer["text"] = self.finder(audio,ID)

root = Tk()
Application(root)
root.title("archangel")
root.mainloop()      

C:\Users\edube\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in true_divide
C:\Users\edube\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in less


In [ ]:
fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6), (ax7, ax8)) = plt.subplots(4, 2)
ax1.plot(AVB_PS['Eduardo0'])
ax2.plot(AVB_PS['Eduardo1'], 'tab:orange')
ax3.plot(AVB_PS['Eduardo2'], 'tab:green')
ax4.plot(AVB_PS['Eduardo3'], 'tab:red')
ax5.plot(AVB_PS['Eduardo0test'])
ax6.plot(AVB_PS['Eduardo1test'], 'tab:orange')
ax7.plot(AVB_PS['Eduardo2test'], 'tab:green')
ax8.plot(AVB_PS['Eduardo3test'], 'tab:red')

for ax in fig.get_axes():
    ax.label_outer()

In [ ]:
fig, ((ax1,ax2)) = plt.subplots(2,1)

ax1.plot(AVB_RS['Eduardo'])
ax2.plot(AVB_RS['Eduardotest'])

In [ ]:
fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6), (ax7, ax8)) = plt.subplots(4, 2)
ax1.plot(AVB_MFCC['Eduardo'][0])
ax2.plot(AVB_MFCC['Eduardo'][1], 'tab:orange')
ax3.plot(AVB_MFCC['Eduardo'][2], 'tab:green')
ax5.plot(AVB_MFCC['Eduardotest'][0])
ax6.plot(AVB_MFCC['Eduardotest'][1], 'tab:orange')
ax7.plot(AVB_MFCC['Eduardotest'][2], 'tab:green')

for ax in fig.get_axes():
    ax.label_outer()